In [3]:
# =========================================================
# XGBoost 기대수명 모델 학습 노트북
# =========================================================

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 데이터 로딩 및 정제
# ---------------------------------------------------------
df = pd.read_csv("./dataset/LifeExpectancyData.csv")
df.columns = df.columns.str.strip()

drop_cols = ['infant deaths', 'thinness 5-9 years', 'Year', 'Population']
df = df.drop(columns=drop_cols)

# ---------------------------------------------------------
# 2. 결측치 처리 (중앙값)
# ---------------------------------------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy="median")
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# ---------------------------------------------------------
# 3. 범주형 인코딩
# ---------------------------------------------------------
df = df.drop("Country", axis=1)
df = pd.get_dummies(df, columns=["Status"], drop_first=True)

# ---------------------------------------------------------
# 4. 데이터 분리
# ---------------------------------------------------------
X = df.drop("Life expectancy", axis=1)
y = np.log1p(df["Life expectancy"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---------------------------------------------------------
# 5. 스케일링
# ---------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------------------------------------------
# 6. XGBoost 모델 학습
# ---------------------------------------------------------
model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train_scaled, y_train)

# ---------------------------------------------------------
# 7. 성능 평가
# ---------------------------------------------------------
pred = np.expm1(model.predict(X_test_scaled))
true = np.expm1(y_test)

print("📊 XGBOOST 성능 요약")
print(f"R²   : {r2_score(true, pred):.4f}")
print(f"RMSE : {np.sqrt(mean_squared_error(true, pred)):.4f}")
print(f"MAE  : {mean_absolute_error(true, pred):.4f}")

# ---------------------------------------------------------
# 8. 모델 패키지 저장
# ---------------------------------------------------------
joblib.dump(model, "./models3/xgb_model.pkl")
joblib.dump(scaler, "./models3/scaler.pkl")
joblib.dump(X.columns.tolist(), "./models3/feature_columns.pkl")

print("✅ 저장 완료: xgb_model.pkl, scaler.pkl, feature_columns.pkl")


📊 XGBOOST 성능 요약
R²   : 0.9640
RMSE : 1.7672
MAE  : 1.2532
✅ 저장 완료: xgb_model.pkl, scaler.pkl, feature_columns.pkl
